In [1]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_percentage_error
import numpy as np
from scipy.constants import c, h, k
from scipy.optimize import curve_fit,fsolve
import matplotlib.pyplot as plt
import tifffile as tif
import math
import plotly.graph_objects as go

In [2]:
def getfile(homefolder, experiment_name):
    RAWdata_folder = homefolder + "/" + experiment_name
    os.chdir(RAWdata_folder)
    file_list = list(filter(lambda x: x[-5:] == ".tiff", os.listdir(RAWdata_folder)))
    file_list.sort()
    return file_list
## get data in form (series,x,y,channel)
def get_data(file_list):
    data = []
    for img_series in range(round(len(file_list) / 9)):
        data_betw = []
        for idx_chl in range(8):
            cI = tif.imread(file_list[img_series * 9 + idx_chl])
            data_betw.append(cI)
        data.append(data_betw)
    data = np.transpose(np.asarray(data),(0,2,3,1))
    return np.asarray(data)

In [3]:
## theory function & fit func
def GT(temp,wavelength):
    result_value = param1/(wavelength**5)/(np.exp(param2/(wavelength*temp))-1)*10**(-6)
    return result_value
def fit_F(wl,a,b,t):
    return (a-b*(wl*10**6-0.548)/0.278)*GT(t,wl)

In [4]:
## fix parameter

wl = np.array([0.548, 0.586, 0.628, 0.667, 0.704, 0.743, 0.786, 0.826])*10**(-6)
sens_factor = np.array([0.017547780102410992, 0.04614964550557874, 0.09544836678000658, 0.18681526194692097, 0.2844845925708136, 0.4279415858839935, 0.6172293335697243, 0.8921496437579155])
sens_factor_b = [-0.6252034172081382, -1.2559592939190707, -2.2804279959666185, -2.896486330314393, -3.511054878378452, -2.6478795393421697, -0.496043324899249, 6.119685014350584]
param1 = h * 2 * c ** 2
param2 = h * c / k
emi = 1
## program parameter
homefolder = os.path.dirname(os.path.dirname(os.getcwd()))
experiment_folder = "\V0_bb_1085"


In [7]:
## get data
file_list = getfile(homefolder,experiment_folder)
data = get_data(file_list)
# data = data[:,25:70,10:50,:]
data = data[:,:,:,:]
print(np.shape(data))
noise = data[0,0,0]
print(noise)
fst_try = (data[0,45,45]-noise)*sens_factor+sens_factor_b
plt.plot(wl,fst_try,"--")
print(fst_try)

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: 'G:\\Uni_Files/\\V0_bb_1085'